In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
model_input = "/Users/awaldrop/Desktop/test.csv"

In [23]:
df = pd.read_csv(model_input)
df.head()

,player,position,team,opp,points_actual,salary,points_projected,sdPts_projected
0,Russell Wilson,QB,sea,nor,44.34,6300,20.000795,1.180859
1,Daniel Jones,QB,nyg,tam,39.24,5000,15.572467,1.329659
2,Deshaun Watson,QB,hou,lac,29.84,6400,20.714121,0.888298
3,Jameis Winston,QB,tam,nyg,29.50,5400,20.066271,1.055659
4,Matt Ryan,QB,atl,ind,26.46,5700,19.894050,0.482604


In [58]:
salaries = {}
points = {}
for pos in df.position.unique():
    available_pos = df[df.position == pos]
    salary = list(available_pos[["player","salary"]].set_index("player").to_dict().values())[0]
    point = list(available_pos[["player","points_projected"]].set_index("player").to_dict().values())[0]
    salaries[pos] = salary
    points[pos] = point
    
pos_num_available = {
    "QB": 1,
    "RB": 3,
    "WR": 4,
    "TE": 2,
    "D": 1
}

In [39]:
salaries

{'QB': {'Russell Wilson': 6300,
  'Daniel Jones': 5000,
  'Deshaun Watson': 6400,
  'Jameis Winston': 5400,
  'Matt Ryan': 5700,
  'Kyle Allen': 4000,
  'Jacoby Brissett': 5200,
  'Philip Rivers': 5800,
  'Dak Prescott': 6500,
  'Tom Brady': 6600,
  'Carson Wentz': 5600,
  'Lamar Jackson': 7000,
  'Mitchell Trubisky': 5100,
  'Kyler Murray': 5800,
  'Case Keenum': 4700,
  'Josh Allen': 5900,
  'Marcus Mariota': 5200,
  'Andy Dalton': 5300,
  'Gardner Minshew': 4900,
  'Jared Goff': 6100,
  'Derek Carr': 4900,
  'Teddy Bridgewater': 4700,
  'Mason Rudolph': 4800,
  'Aaron Rodgers': 6100,
  'Kirk Cousins': 5100,
  'Jimmy Garoppolo': 6200,
  'Matthew Stafford': 5500,
  'Baker Mayfield': 6000,
  'Josh Rosen': 4500,
  'Joe Flacco': 4600,
  'Luke Falk': 4600},
 'RB': {'Mark Ingram': 5700,
  'Alvin Kamara': 8000,
  'Christian McCaffrey': 8700,
  'Phillip Lindsay': 4300,
  'Dalvin Cook': 7800,
  'Tony Pollard': 3800,
  'LeSean McCoy': 5000,
  'Ezekiel Elliott': 8900,
  'Rex Burkhead': 3900,
  

In [40]:
points

{'QB': {'Russell Wilson': 20.00079524,
  'Daniel Jones': 15.57246667,
  'Deshaun Watson': 20.71412143,
  'Jameis Winston': 20.06627143,
  'Matt Ryan': 19.89405,
  'Kyle Allen': 15.25246667,
  'Jacoby Brissett': 16.8983,
  'Philip Rivers': 19.25102857,
  'Dak Prescott': 23.39396429,
  'Tom Brady': 19.85049286,
  'Carson Wentz': 19.62728571,
  'Lamar Jackson': 22.49712143,
  'Mitchell Trubisky': 16.2735,
  'Kyler Murray': 18.95134286,
  'Case Keenum': 13.64516667,
  'Josh Allen': 20.50398571,
  'Marcus Mariota': 14.3648,
  'Andy Dalton': 15.76299524,
  'Gardner Minshew': 13.476270000000001,
  'Jared Goff': 17.44192857,
  'Derek Carr': 13.51606667,
  'Teddy Bridgewater': 13.81636667,
  'Mason Rudolph': 15.683470000000002,
  'Aaron Rodgers': 19.4457,
  'Kirk Cousins': 17.93794762,
  'Jimmy Garoppolo': 17.68062857,
  'Matthew Stafford': 17.65968571,
  'Baker Mayfield': 17.72174286,
  'Josh Rosen': 10.59316667,
  'Joe Flacco': 13.75786667,
  'Luke Falk': 9.23983},
 'RB': {'Mark Ingram': 13.1

In [41]:
from pulp import *

In [59]:
_vars = {k: LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}

In [60]:
_vars

{'QB': {'Russell Wilson': QB_Russell_Wilson,
  'Daniel Jones': QB_Daniel_Jones,
  'Deshaun Watson': QB_Deshaun_Watson,
  'Jameis Winston': QB_Jameis_Winston,
  'Matt Ryan': QB_Matt_Ryan,
  'Kyle Allen': QB_Kyle_Allen,
  'Jacoby Brissett': QB_Jacoby_Brissett,
  'Philip Rivers': QB_Philip_Rivers,
  'Dak Prescott': QB_Dak_Prescott,
  'Tom Brady': QB_Tom_Brady,
  'Carson Wentz': QB_Carson_Wentz,
  'Lamar Jackson': QB_Lamar_Jackson,
  'Mitchell Trubisky': QB_Mitchell_Trubisky,
  'Kyler Murray': QB_Kyler_Murray,
  'Case Keenum': QB_Case_Keenum,
  'Josh Allen': QB_Josh_Allen,
  'Marcus Mariota': QB_Marcus_Mariota,
  'Andy Dalton': QB_Andy_Dalton,
  'Gardner Minshew': QB_Gardner_Minshew,
  'Jared Goff': QB_Jared_Goff,
  'Derek Carr': QB_Derek_Carr,
  'Teddy Bridgewater': QB_Teddy_Bridgewater,
  'Mason Rudolph': QB_Mason_Rudolph,
  'Aaron Rodgers': QB_Aaron_Rodgers,
  'Kirk Cousins': QB_Kirk_Cousins,
  'Jimmy Garoppolo': QB_Jimmy_Garoppolo,
  'Matthew Stafford': QB_Matthew_Stafford,
  'Baker Ma

In [70]:
SALARY_CAP = 50000
max_players = 8

In [80]:
prob = LpProblem("Fantasy", LpMaximize)
rewards = []
costs = []
total_players = []
position_constraints = []

# Setting up the reward
for k, v in _vars.items():
    costs += lpSum([salaries[k][i] * _vars[k][i] for i in v])
    rewards += lpSum([points[k][i] * _vars[k][i] for i in v])
    total_players += lpSum([_vars[k][i] for i in v])
    prob += lpSum([_vars[k][i] for i in v]) <= pos_num_available[k]
    prob += lpSum([_vars[k][i] for i in v]) >= 1
    #prob += lpSum([pos_flex[k] * _vars[k][i] for i in v]) <= pos_flex_available
    
prob += lpSum(rewards)
prob += lpSum(costs) <= SALARY_CAP
prob += lpSum(total_players) <= max_players
prob += lpSum(total_players) >= max_players

In [81]:
prob

Fantasy:
MAXIMIZE
7.846333333*D_49ers + 7.647*D_Bears + 6.271*D_Bengals + 8.055857142999999*D_Bills + 5.8826666670000005*D_Broncos + 5.8870000000000005*D_Browns + 6.462999999999999*D_Buccaneers + 7.141666667000001*D_Cardinals + 7.134285714*D_Chargers + 6.619666667000001*D_Chiefs + 6.7582*D_Colts + 9.112571428999999*D_Cowboys + 6.101714286*D_Eagles + 5.7066*D_Falcons + 6.6974*D_Giants + 7.586*D_Jaguars + 6.0672*D_Lions + 7.522142857*D_Packers + 7.1192*D_Panthers + 12.26471429*D_Patriots + 5.2846*D_Raiders + 7.065571428999999*D_Rams + 5.319333333*D_Ravens + 6.1256*D_Redskins + 6.807666667*D_Saints + 6.743714286*D_Seahawks + 6.9094*D_Steelers + 6.251*D_Texans + 8.933428571*D_Titans + 8.229285714*D_Vikings + 19.4457*QB_Aaron_Rodgers + 15.76299524*QB_Andy_Dalton + 17.72174286*QB_Baker_Mayfield + 19.62728571*QB_Carson_Wentz + 13.64516667*QB_Case_Keenum + 23.39396429*QB_Dak_Prescott + 15.57246667*QB_Daniel_Jones + 13.51606667*QB_Derek_Carr + 20.71412143*QB_Deshaun_Watson + 13.476270000000001*

In [82]:
prob.solve()

1

In [73]:
def summary(prob):
    div = '---------------------------------------\n'
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
    print(div)
    print("Constraints:")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))

In [83]:
summary(prob)

Variables:

D_Patriots = 1.0
QB_Dak_Prescott = 1.0
RB_Austin_Ekeler = 1.0
RB_Christian_McCaffrey = 1.0
TE_Zach_Ertz = 1.0
WR_Emmanuel_Sanders = 1.0
WR_Julian_Edelman = 1.0
WR_Keenan_Allen = 1.0
---------------------------------------

Constraints:
3800*1.0 + 6500*1.0 + 7200*1.0 + 8700*1.0 + 5700*1.0 + 4800*1.0 + 6300*1.0 + 7000*1.0 = 50000.0
---------------------------------------

Score:
12.26471429*1.0 + 23.39396429*1.0 + 20.90333333*1.0 + 26.06475*1.0 + 17.364375*1.0 + 16.4265*1.0 + 18.808*1.0 + 20.63158333*1.0 = 155.85722024


In [99]:
def output_performance(prob, df):
    player_names = []
    for v in prob.variables():
        if v.varValue != 0:
            player_names.append(" ".join(v.name.split("_")[1:]))

    score_df = df[df.player.isin(player_names)]
    print("Actual week score: {0}".format(score_df.points_actual.sum()))
    return score_df

In [100]:
output_performance(prob, df)

Actual week score: 162.64


,player,position,team,opp,points_actual,salary,points_projected,sdPts_projected
8,Dak Prescott,QB,dal,mia,23.54,6500,23.393964,2.209696
33,Christian McCaffrey,RB,car,ari,30.80,8700,26.064750,1.428647
47,Austin Ekeler,RB,lac,hou,15.10,7200,20.903333,1.726666
90,Keenan Allen,WR,lac,hou,46.60,7000,20.631583,2.736726
104,Julian Edelman,WR,nwe,nyj,19.20,6300,18.808000,2.500448
162,Emmanuel Sanders,WR,den,gnb,3.00,4800,16.426500,2.958249
181,Zach Ertz,TE,phi,det,10.40,5700,17.364375,2.663153
202,Patriots,D,nwe,nyj,14.00,3800,12.264714,1.954914
